<a href="https://colab.research.google.com/github/pietroperrone01/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

In [5]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

# Part 1: loading data

In [14]:
# this is for getting rid of edges (usually artifacted)
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

# define Svalbard area
svalbard_geom = ee.Geometry.Polygon(
    [[-2.434071, 81.099918],
     [46.394056, 81.099918],
     [46.394056, 74.304052],
     [-2.434071, 74.304052],
     [-2.434071, 81.099918]]
)

In [47]:
# Sentinel-1 collection for Svalbard
svalbard_s1 = {}
polarizations = ['HH', 'VV', 'HV', 'VH']

for pol in polarizations:
    svalbard_s1[pol] = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select(pol)
    .map(mask_edge)
    )


In [50]:
svalbard_s1HH = svalbard_s1['HH']
svalbard_s1VV = svalbard_s1['VV']
svalbard_s1HV = svalbard_s1['HV']
svalbard_s1VH = svalbard_s1['VH']

In [39]:
# time ranges
spring = ee.Filter.date('2024-03-01', '2024-04-20')
late_spring = ee.Filter.date('2024-04-21', '2024-06-10')
summer = ee.Filter.date('2024-06-11', '2024-08-31')

In [51]:
Map = geemap.Map()
Map.centerObject(svalbard_geom, zoom=6)
Map.addLayer(svalbard_s1HH, {'min': -25, 'max': 5}, 'Sentinel-1') # display a certain range of dB
Map

Map(center=[78.48364888332274, 21.979992499999994], controls=(WidgetControl(options=['position', 'transparent_…

In [46]:
svalbard_s1hh_spring = svalbard_s1HH.filter(spring)

Spring_Map = geemap.Map()
Spring_Map.centerObject(svalbard_geom, zoom=6)

# check if the collection is empty
if svalbard_s1hh_spring.size().getInfo() > 0:
    # mosaic
    spring_mosaic = svalbard_s1HH_spring.mosaic()
    Spring_Map.addLayer(spring_mosaic, {'min': -25, 'max': 5}, 'Sentinel-1 spring mosaic')
    Spring_Map
else:
    print("No Sentinel-1 images found for the spring period in Svalbard with the specified filters.")
    Spring_Map

Spring_Map

Map(center=[78.48364888332274, 21.979992499999994], controls=(WidgetControl(options=['position', 'transparent_…